# 第15章: 動的因果効果の推定 (Estimation of Dynamic Causal Effects)

この章では、時系列データにおける動的因果効果の推定について学びます。

## 主なトピック
- 動的因果効果とは
- 分布ラグモデル（Distributed Lag Models）
- 外生性と厳密外生性
- インパルス応答関数

## 必要なパッケージ

In [ ]:
# 必要なパッケージのインストールと読み込み
if (!require("AER")) install.packages("AER")
if (!require("dynlm")) install.packages("dynlm")
if (!require("lmtest")) install.packages("lmtest")
if (!require("sandwich")) install.packages("sandwich")
if (!require("vars")) install.packages("vars")

library(AER)
library(dynlm)
library(lmtest)
library(sandwich)
library(vars)

---
## 問題1: 動的因果効果の概念

政策変数 $x_t$ がアウトカム $y_t$ に与える影響が時間を通じて広がる場合を考えます。

以下のシミュレーションでデータを生成し、動的因果効果を推定します。

```r
set.seed(123)
n <- 200
x <- rnorm(n)  # 外生的なショック
# 真の動的因果効果: 即時効果0.5, 1期ラグ0.3, 2期ラグ0.1
y <- numeric(n)
for (t in 3:n) {
  y[t] <- 0.5 * x[t] + 0.3 * x[t-1] + 0.1 * x[t-2] + rnorm(1, 0, 0.5)
}
dyn_data <- ts(cbind(y = y, x = x))
```

1. 分布ラグモデル DL(2) を推定してください
2. 推定された係数と真の値を比較してください
3. 累積乗数を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(123)
n <- 200
x <- rnorm(n)
y <- numeric(n)
for (t in 3:n) {
  y[t] <- 0.5 * x[t] + 0.3 * x[t-1] + 0.1 * x[t-2] + rnorm(1, 0, 0.5)
}
dyn_data <- ts(cbind(y = y, x = x))

# 1. DL(2)モデルの推定
dl2_model <- dynlm(y ~ x + L(x, 1) + L(x, 2), data = dyn_data)
cat("DL(2)モデルの推定結果:\n")
summary(dl2_model)

# 2. 推定値と真の値の比較
true_values <- c(0.5, 0.3, 0.1)
estimated <- coef(dl2_model)[c("x", "L(x, 1)", "L(x, 2)")]

comparison <- data.frame(
  ラグ = c(0, 1, 2),
  真の値 = true_values,
  推定値 = estimated
)
cat("\n係数の比較:\n")
print(comparison)

# 3. 累積乗数の計算
cumulative <- cumsum(estimated)
cat("\n累積乗数:\n")
cat("0期: ", cumulative[1], "\n")
cat("1期まで:", cumulative[2], "\n")
cat("2期まで（長期乗数）:", cumulative[3], "\n")

cat("\n真の累積乗数:", cumsum(true_values), "\n")

---
## 問題2: ラグ次数の選択

動的効果を推定する際、何期のラグを含めるべきかを決定する必要があります。

```r
set.seed(456)
n <- 300
x <- rnorm(n)
y <- numeric(n)
# 真のラグは3期まで
for (t in 4:n) {
  y[t] <- 0.4 * x[t] + 0.3 * x[t-1] + 0.2 * x[t-2] + 0.1 * x[t-3] + rnorm(1, 0, 0.5)
}
lag_data <- ts(cbind(y = y, x = x))
```

1. DL(1)からDL(5)まで推定し、情報量基準を比較してください
2. F検定を使ってラグの追加が有意かどうか検定してください
3. 最適なラグ次数を決定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(456)
n <- 300
x <- rnorm(n)
y <- numeric(n)
for (t in 4:n) {
  y[t] <- 0.4 * x[t] + 0.3 * x[t-1] + 0.2 * x[t-2] + 0.1 * x[t-3] + rnorm(1, 0, 0.5)
}
lag_data <- ts(cbind(y = y, x = x))

# 1. 異なるラグ次数でのモデル推定
results <- data.frame(ラグ次数 = 1:5, AIC = NA, BIC = NA, R2_adj = NA)

models <- list()
for (p in 1:5) {
  formula <- as.formula(paste("y ~ x +", paste(paste0("L(x, ", 1:p, ")"), collapse = " + ")))
  models[[p]] <- dynlm(formula, data = lag_data)
  results$AIC[p] <- AIC(models[[p]])
  results$BIC[p] <- BIC(models[[p]])
  results$R2_adj[p] <- summary(models[[p]])$adj.r.squared
}

cat("情報量基準の比較:\n")
print(results)

# 2. F検定によるラグの追加の検定
cat("\nF検定によるラグの追加の検定:\n")
for (p in 2:5) {
  f_test <- anova(models[[p-1]], models[[p]])
  cat(paste("DL(", p-1, ") vs DL(", p, "): F = ", 
            round(f_test$F[2], 3), ", p値 = ", 
            round(f_test$`Pr(>F)`[2], 4), "\n", sep = ""))
}

# 3. 最適なラグ次数の決定
cat("\n最適なラグ次数:\n")
cat("AIC最小:", which.min(results$AIC), "\n")
cat("BIC最小:", which.min(results$BIC), "\n")

# 選択されたモデルの推定結果
cat("\nDL(3)モデルの推定結果:\n")
summary(models[[3]])

---
## 問題3: ADL（自己回帰分布ラグ）モデル

被説明変数のラグも含めたADLモデルを推定します。

```r
set.seed(789)
n <- 200
x <- rnorm(n)
y <- numeric(n)
y[1] <- rnorm(1)
# ADL(1,1)モデル
for (t in 2:n) {
  y[t] <- 0.4 * y[t-1] + 0.5 * x[t] + 0.2 * x[t-1] + rnorm(1, 0, 0.5)
}
adl_data <- ts(cbind(y = y, x = x))
```

1. ADL(1,1)モデルを推定してください
2. 衝撃乗数と長期乗数を計算してください
3. 動的乗数を10期先まで計算してグラフ化してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(789)
n <- 200
x <- rnorm(n)
y <- numeric(n)
y[1] <- rnorm(1)
for (t in 2:n) {
  y[t] <- 0.4 * y[t-1] + 0.5 * x[t] + 0.2 * x[t-1] + rnorm(1, 0, 0.5)
}
adl_data <- ts(cbind(y = y, x = x))

# 1. ADL(1,1)モデルの推定
adl_model <- dynlm(y ~ L(y, 1) + x + L(x, 1), data = adl_data)
cat("ADL(1,1)モデルの推定結果:\n")
summary(adl_model)

# 係数の抽出
rho <- coef(adl_model)["L(y, 1)"]
beta0 <- coef(adl_model)["x"]
beta1 <- coef(adl_model)["L(x, 1)"]

# 2. 衝撃乗数と長期乗数
impact_multiplier <- beta0
long_run_multiplier <- (beta0 + beta1) / (1 - rho)

cat("\n乗数:\n")
cat("衝撃乗数（即時効果）:", impact_multiplier, "\n")
cat("長期乗数:", long_run_multiplier, "\n")

cat("\n真の値:\n")
cat("衝撃乗数: 0.5\n")
cat("長期乗数: (0.5 + 0.2) / (1 - 0.4) =", (0.5 + 0.2) / (1 - 0.4), "\n")

# 3. 動的乗数の計算
horizons <- 10
dynamic_multiplier <- numeric(horizons + 1)
dynamic_multiplier[1] <- beta0  # h = 0
dynamic_multiplier[2] <- rho * beta0 + beta1  # h = 1

for (h in 3:(horizons + 1)) {
  dynamic_multiplier[h] <- rho * dynamic_multiplier[h - 1]
}

cumulative_multiplier <- cumsum(dynamic_multiplier)

# 結果の表示
cat("\n動的乗数:\n")
print(data.frame(
  期間 = 0:horizons,
  動的乗数 = round(dynamic_multiplier, 4),
  累積乗数 = round(cumulative_multiplier, 4)
))

# グラフ
par(mfrow = c(1, 2))
barplot(dynamic_multiplier, names.arg = 0:horizons, 
        main = "動的乗数", xlab = "期間", ylab = "乗数")
plot(0:horizons, cumulative_multiplier, type = "b", pch = 16,
     main = "累積乗数", xlab = "期間", ylab = "累積")
abline(h = long_run_multiplier, lty = 2, col = "red")
legend("bottomright", legend = "長期乗数", lty = 2, col = "red")
par(mfrow = c(1, 1))

---
## 問題4: 外生性の仮定

動的因果効果の推定には説明変数の外生性が重要です。

```r
set.seed(111)
n <- 200

# ケース1: xが厳密外生的
x1 <- rnorm(n)
e1 <- rnorm(n, 0, 0.5)
y1 <- numeric(n)
for (t in 2:n) {
  y1[t] <- 0.5 * x1[t] + 0.3 * x1[t-1] + e1[t]
}

# ケース2: xがyの過去に影響を受ける（厳密外生性の違反）
x2 <- numeric(n)
y2 <- numeric(n)
x2[1] <- rnorm(1)
y2[1] <- rnorm(1)
for (t in 2:n) {
  x2[t] <- 0.3 * y2[t-1] + rnorm(1)  # フィードバック
  y2[t] <- 0.5 * x2[t] + 0.3 * x2[t-1] + rnorm(1, 0, 0.5)
}
```

1. 両方のケースでDL(1)モデルを推定してください
2. 推定結果を比較し、バイアスがあるか確認してください
3. グレンジャー因果性検定を行ってください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(111)
n <- 200

# ケース1: 厳密外生的
x1 <- rnorm(n)
e1 <- rnorm(n, 0, 0.5)
y1 <- numeric(n)
for (t in 2:n) {
  y1[t] <- 0.5 * x1[t] + 0.3 * x1[t-1] + e1[t]
}
data1 <- ts(cbind(y = y1, x = x1))

# ケース2: フィードバックあり
x2 <- numeric(n)
y2 <- numeric(n)
x2[1] <- rnorm(1)
y2[1] <- rnorm(1)
for (t in 2:n) {
  x2[t] <- 0.3 * y2[t-1] + rnorm(1)
  y2[t] <- 0.5 * x2[t] + 0.3 * x2[t-1] + rnorm(1, 0, 0.5)
}
data2 <- ts(cbind(y = y2, x = x2))

# 1. DL(1)モデルの推定
model1 <- dynlm(y ~ x + L(x, 1), data = data1)
model2 <- dynlm(y ~ x + L(x, 1), data = data2)

cat("ケース1（厳密外生）:\n")
print(coef(model1)[c("x", "L(x, 1)")])

cat("\nケース2（フィードバックあり）:\n")
print(coef(model2)[c("x", "L(x, 1)")])

# 2. 比較
cat("\n真の係数との比較:\n")
comparison <- data.frame(
  係数 = c("β0", "β1"),
  真の値 = c(0.5, 0.3),
  ケース1 = coef(model1)[c("x", "L(x, 1)")],
  ケース2 = coef(model2)[c("x", "L(x, 1)")]
)
print(comparison)

cat("\nケース2ではフィードバックによりバイアスが生じる可能性があります。\n")

# 3. グレンジャー因果性検定
cat("\nグレンジャー因果性検定（y → x）:\n")

# ケース1
granger1_full <- dynlm(x ~ L(x, 1:2) + L(y, 1:2), data = data1)
granger1_rest <- dynlm(x ~ L(x, 1:2), data = data1)
cat("ケース1:\n")
print(anova(granger1_rest, granger1_full))

# ケース2
granger2_full <- dynlm(x ~ L(x, 1:2) + L(y, 1:2), data = data2)
granger2_rest <- dynlm(x ~ L(x, 1:2), data = data2)
cat("\nケース2:\n")
print(anova(granger2_rest, granger2_full))

cat("\n解釈:\n")
cat("ケース2ではyからxへの因果関係が検出されます（厳密外生性の違反）。\n")

---
## 問題5: HAC標準誤差を使った推論

時系列データでは系列相関があるため、HAC標準誤差を使用します。

```r
set.seed(222)
n <- 150
x <- rnorm(n)
# 誤差項にAR(1)過程
e <- arima.sim(n = n, model = list(ar = 0.5), sd = 0.5)
y <- numeric(n)
for (t in 2:n) {
  y[t] <- 0.6 * x[t] + 0.4 * x[t-1] + e[t]
}
hac_data <- ts(cbind(y = y, x = x))
```

1. DL(1)モデルを推定してください
2. 通常の標準誤差とHAC標準誤差を比較してください
3. 長期乗数の標準誤差をデルタ法で計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(222)
n <- 150
x <- rnorm(n)
e <- arima.sim(n = n, model = list(ar = 0.5), sd = 0.5)
y <- numeric(n)
for (t in 2:n) {
  y[t] <- 0.6 * x[t] + 0.4 * x[t-1] + e[t]
}
hac_data <- ts(cbind(y = y, x = x))

# 1. DL(1)モデルの推定
dl1_model <- dynlm(y ~ x + L(x, 1), data = hac_data)
cat("DL(1)モデルの推定結果:\n")
summary(dl1_model)

# 2. 標準誤差の比較
cat("\n通常の標準誤差:\n")
print(coeftest(dl1_model))

cat("\nHAC標準誤差（Newey-West）:\n")
hac_test <- coeftest(dl1_model, vcov = NeweyWest(dl1_model))
print(hac_test)

# 比較表
se_comparison <- data.frame(
  係数 = names(coef(dl1_model)),
  通常SE = summary(dl1_model)$coefficients[, "Std. Error"],
  HAC_SE = hac_test[, "Std. Error"]
)
se_comparison$比率 <- se_comparison$HAC_SE / se_comparison$通常SE
cat("\n標準誤差の比較:\n")
print(se_comparison)

# 3. 長期乗数の標準誤差（デルタ法）
# DL(1)モデルでは長期乗数 = β0 + β1
beta <- coef(dl1_model)[c("x", "L(x, 1)")]
long_run <- sum(beta)

# HAC共分散行列
V_hac <- NeweyWest(dl1_model)[c("x", "L(x, 1)"), c("x", "L(x, 1)")]

# デルタ法: Var(β0 + β1) = Var(β0) + Var(β1) + 2*Cov(β0, β1)
gradient <- c(1, 1)  # ∂(β0 + β1)/∂β
se_long_run <- sqrt(t(gradient) %*% V_hac %*% gradient)

cat("\n長期乗数の推定:\n")
cat("推定値:", long_run, "\n")
cat("HAC標準誤差:", se_long_run, "\n")
cat("95%信頼区間:", long_run - 1.96 * se_long_run, "から", 
    long_run + 1.96 * se_long_run, "\n")
cat("\n真の長期乗数: 0.6 + 0.4 = 1.0\n")

---
## 問題6: オレンジジュースと凍結気温

フロリダの凍結気温がオレンジジュース価格に与える動的影響を分析します。

以下のシミュレーションデータを使います。

```r
set.seed(333)
n <- 120  # 10年間の月次データ
# 凍結日数（冬に多い）
month <- rep(1:12, 10)
freeze <- ifelse(month %in% c(12, 1, 2, 3), rpois(n, 3), rpois(n, 0.1))
freeze <- freeze[1:n]

# 価格変化（凍結の影響は0-2期まで）
price_change <- numeric(n)
for (t in 3:n) {
  price_change[t] <- 2 * freeze[t] + 1.5 * freeze[t-1] + 0.5 * freeze[t-2] + rnorm(1, 0, 3)
}
oj_data <- ts(cbind(price_change = price_change, freeze = freeze), frequency = 12)
```

1. データの時系列プロットを作成してください
2. DL(4)モデルを推定してください
3. 凍結による価格への累積効果を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(333)
n <- 120
month <- rep(1:12, 10)
freeze <- ifelse(month %in% c(12, 1, 2, 3), rpois(n, 3), rpois(n, 0.1))
freeze <- freeze[1:n]

price_change <- numeric(n)
for (t in 3:n) {
  price_change[t] <- 2 * freeze[t] + 1.5 * freeze[t-1] + 0.5 * freeze[t-2] + rnorm(1, 0, 3)
}
oj_data <- ts(cbind(price_change = price_change, freeze = freeze), frequency = 12)

# 1. 時系列プロット
par(mfrow = c(2, 1))
plot(oj_data[, "freeze"], main = "凍結日数", ylab = "日数", xlab = "時間")
plot(oj_data[, "price_change"], main = "価格変化", ylab = "%", xlab = "時間")
par(mfrow = c(1, 1))

# 2. DL(4)モデルの推定
dl4_oj <- dynlm(price_change ~ freeze + L(freeze, 1) + L(freeze, 2) + 
                L(freeze, 3) + L(freeze, 4), data = oj_data)
cat("DL(4)モデルの推定結果:\n")
summary(dl4_oj)

# HAC標準誤差
cat("\nHAC標準誤差を使用した結果:\n")
print(coeftest(dl4_oj, vcov = NeweyWest(dl4_oj)))

# 3. 累積効果の計算
coefs <- coef(dl4_oj)[c("freeze", "L(freeze, 1)", "L(freeze, 2)", 
                        "L(freeze, 3)", "L(freeze, 4)")]
cumulative <- cumsum(coefs)

cat("\n累積効果:\n")
print(data.frame(
  ラグ = 0:4,
  動的乗数 = round(coefs, 3),
  累積乗数 = round(cumulative, 3)
))

# グラフ
par(mfrow = c(1, 2))
barplot(coefs, names.arg = 0:4, main = "動的乗数", 
        xlab = "ラグ（月）", ylab = "価格への影響（%）")
plot(0:4, cumulative, type = "b", pch = 16, main = "累積乗数",
     xlab = "ラグ（月）", ylab = "累積効果（%）")
par(mfrow = c(1, 1))

cat("\n解釈:\n")
cat("凍結日数が1日増えると、価格は即座に約", round(coefs[1], 2), "%上昇し、\n")
cat("2ヶ月後までの累積効果は約", round(cumulative[3], 2), "%になります。\n")

---
## 問題7: 金融政策の効果

金利変更がインフレ率に与える動的効果を分析します。

```r
set.seed(444)
n <- 160  # 40年間の四半期データ

# 金利変更（外生的なショック）
rate_change <- rnorm(n, 0, 0.5)

# インフレ率（金利上昇は遅れてインフレを抑制）
inflation <- numeric(n)
inflation[1:4] <- rnorm(4, 2, 0.5)
for (t in 5:n) {
  inflation[t] <- 0.7 * inflation[t-1] - 0.2 * rate_change[t-2] - 
                  0.3 * rate_change[t-3] - 0.2 * rate_change[t-4] + 
                  rnorm(1, 0, 0.3)
}
macro_data <- ts(cbind(inflation = inflation, rate = rate_change), frequency = 4)
```

1. ADL(1,4)モデルを推定してください
2. インパルス応答関数を計算してください
3. 金利1%上昇の累積効果を推定してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(444)
n <- 160
rate_change <- rnorm(n, 0, 0.5)

inflation <- numeric(n)
inflation[1:4] <- rnorm(4, 2, 0.5)
for (t in 5:n) {
  inflation[t] <- 0.7 * inflation[t-1] - 0.2 * rate_change[t-2] - 
                  0.3 * rate_change[t-3] - 0.2 * rate_change[t-4] + 
                  rnorm(1, 0, 0.3)
}
macro_data <- ts(cbind(inflation = inflation, rate = rate_change), frequency = 4)

# 1. ADL(1,4)モデルの推定
adl_macro <- dynlm(inflation ~ L(inflation, 1) + rate + L(rate, 1:4), data = macro_data)
cat("ADL(1,4)モデルの推定結果:\n")
summary(adl_macro)

# 係数の抽出
rho <- coef(adl_macro)["L(inflation, 1)"]
beta <- coef(adl_macro)[c("rate", "L(rate, 1:4)1", "L(rate, 1:4)2", 
                          "L(rate, 1:4)3", "L(rate, 1:4)4")]

# 2. インパルス応答関数の計算
horizons <- 20
irf <- numeric(horizons + 1)

# 最初の5期はβ係数
for (h in 0:4) {
  irf[h + 1] <- beta[h + 1]
}

# それ以降はρによる減衰
for (h in 5:horizons) {
  irf[h + 1] <- rho * irf[h]  # h-1期の応答 × ρ
}

# 累積応答
cumulative_irf <- cumsum(irf)

# 結果の表示
cat("\nインパルス応答関数（最初の10期）:\n")
print(data.frame(
  期間 = 0:10,
  IRF = round(irf[1:11], 4),
  累積IRF = round(cumulative_irf[1:11], 4)
))

# 3. 長期累積効果
long_run_effect <- sum(beta) / (1 - rho)
cat("\n長期累積効果:", round(long_run_effect, 4), "\n")

# グラフ
par(mfrow = c(1, 2))
plot(0:horizons, irf, type = "h", lwd = 2, main = "インパルス応答関数",
     xlab = "四半期", ylab = "インフレ率への影響")
abline(h = 0, lty = 2)

plot(0:horizons, cumulative_irf, type = "l", lwd = 2, main = "累積インパルス応答",
     xlab = "四半期", ylab = "累積効果")
abline(h = long_run_effect, lty = 2, col = "red")
legend("bottomright", legend = "長期効果", lty = 2, col = "red")
par(mfrow = c(1, 1))

cat("\n解釈:\n")
cat("金利1%の上昇は、即座にはインフレに影響を与えませんが、\n")
cat("2-4四半期後にインフレを低下させ、長期的には約", 
    round(long_run_effect, 2), "%の低下をもたらします。\n")

---
## 問題8: VARモデルによる動的分析

複数の変数間の動的関係をVAR（Vector Autoregression）モデルで分析します。

```r
set.seed(555)
n <- 200

# 2変数VARシステム
y1 <- numeric(n)
y2 <- numeric(n)
y1[1] <- rnorm(1)
y2[1] <- rnorm(1)

for (t in 2:n) {
  y1[t] <- 0.5 * y1[t-1] + 0.2 * y2[t-1] + rnorm(1, 0, 0.5)
  y2[t] <- 0.3 * y1[t-1] + 0.4 * y2[t-1] + rnorm(1, 0, 0.5)
}

var_data <- cbind(y1 = y1, y2 = y2)
```

1. VAR(1)モデルを推定してください
2. グレンジャー因果性を検定してください
3. インパルス応答関数を計算してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# データの生成
set.seed(555)
n <- 200

y1 <- numeric(n)
y2 <- numeric(n)
y1[1] <- rnorm(1)
y2[1] <- rnorm(1)

for (t in 2:n) {
  y1[t] <- 0.5 * y1[t-1] + 0.2 * y2[t-1] + rnorm(1, 0, 0.5)
  y2[t] <- 0.3 * y1[t-1] + 0.4 * y2[t-1] + rnorm(1, 0, 0.5)
}

var_data <- cbind(y1 = y1, y2 = y2)

# 1. VAR(1)モデルの推定
var_model <- VAR(var_data, p = 1)
cat("VAR(1)モデルの推定結果:\n")
summary(var_model)

# 係数行列の抽出
cat("\n係数行列:\n")
coef_matrix <- rbind(
  coef(var_model$varresult$y1)[1:2],
  coef(var_model$varresult$y2)[1:2]
)
rownames(coef_matrix) <- c("y1", "y2")
colnames(coef_matrix) <- c("y1.l1", "y2.l1")
print(round(coef_matrix, 3))

cat("\n真の係数行列:\n")
print(matrix(c(0.5, 0.3, 0.2, 0.4), nrow = 2, byrow = TRUE,
             dimnames = list(c("y1", "y2"), c("y1.l1", "y2.l1"))))

# 2. グレンジャー因果性検定
cat("\nグレンジャー因果性検定:\n")
cat("\ny2 → y1（y2がy1をグレンジャー因果するか）:\n")
print(causality(var_model, cause = "y2")$Granger)

cat("\ny1 → y2（y1がy2をグレンジャー因果するか）:\n")
print(causality(var_model, cause = "y1")$Granger)

# 3. インパルス応答関数
irf_results <- irf(var_model, n.ahead = 15, boot = TRUE, ci = 0.95)

# プロット
plot(irf_results)

cat("\n解釈:\n")
cat("両方向のグレンジャー因果関係が検出されます。\n")
cat("y1とy2は相互に影響を与え合っています。\n")

---
## 問題9: 予測誤差分散分解

VARモデルを使って、各変数の変動がどの程度他の変数のショックで説明されるかを分析します。

問題8のデータを使って：

1. 予測誤差分散分解を計算してください
2. 各時点での分散の構成を確認してください
3. 結果を解釈してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# 問題8のデータとモデルを再利用
set.seed(555)
n <- 200

y1 <- numeric(n)
y2 <- numeric(n)
y1[1] <- rnorm(1)
y2[1] <- rnorm(1)

for (t in 2:n) {
  y1[t] <- 0.5 * y1[t-1] + 0.2 * y2[t-1] + rnorm(1, 0, 0.5)
  y2[t] <- 0.3 * y1[t-1] + 0.4 * y2[t-1] + rnorm(1, 0, 0.5)
}

var_data <- cbind(y1 = y1, y2 = y2)
var_model <- VAR(var_data, p = 1)

# 1. 予測誤差分散分解
fevd_results <- fevd(var_model, n.ahead = 15)

# 2. 結果の表示
cat("y1の予測誤差分散分解:\n")
print(round(fevd_results$y1, 4))

cat("\ny2の予測誤差分散分解:\n")
print(round(fevd_results$y2, 4))

# グラフ
plot(fevd_results)

# 3. 解釈
cat("\n解釈:\n")
cat("y1の変動:\n")
cat("- 短期では主に自身のショックで説明される\n")
cat("- 時間が経つとy2のショックの寄与も増加\n")
cat("- 長期ではy2のショックが約", 
    round(fevd_results$y1[15, "y2"] * 100, 1), "%を説明\n\n")

cat("y2の変動:\n")
cat("- y1のショックが比較的大きな影響を持つ\n")
cat("- 長期ではy1のショックが約", 
    round(fevd_results$y2[15, "y1"] * 100, 1), "%を説明\n")

---
## 問題10: 実データでの動的因果効果の推定

USMacroGデータを使って、金利とインフレの動的関係を分析します。

1. データを読み込み、インフレ率と3ヶ月国債金利を取得してください
2. VARモデルの最適なラグ次数を選択してください
3. グレンジャー因果性検定とインパルス応答を分析してください

In [ ]:
# ここに回答を入力してください


### 模範解答

In [ ]:
# 1. データの読み込みと準備
data("USMacroG")

# インフレ率（CPIの対数差分）と3ヶ月国債金利
inflation <- diff(log(USMacroG[, "cpi"])) * 400  # 年率換算
tbill <- USMacroG[-1, "tbill"]  # 差分を取ったのでサイズを合わせる

macro_ts <- cbind(inflation = as.numeric(inflation), tbill = as.numeric(tbill))
macro_ts <- macro_ts[complete.cases(macro_ts), ]

# データの確認
par(mfrow = c(2, 1))
plot(inflation, main = "インフレ率（年率）", ylab = "%")
plot(tbill, main = "3ヶ月国債金利", ylab = "%")
par(mfrow = c(1, 1))

# 2. ラグ次数の選択
lag_select <- VARselect(macro_ts, lag.max = 8)
cat("ラグ次数選択基準:\n")
print(lag_select$selection)
print(lag_select$criteria)

# 選択されたラグでVARを推定
optimal_lag <- lag_select$selection["SC(n)"]  # BIC基準
var_macro <- VAR(macro_ts, p = optimal_lag)

cat("\nVAR(", optimal_lag, ")モデルの推定結果:\n", sep = "")
summary(var_macro)

# 3. グレンジャー因果性検定
cat("\nグレンジャー因果性検定:\n")
cat("\n金利 → インフレ:\n")
print(causality(var_macro, cause = "tbill")$Granger)

cat("\nインフレ → 金利:\n")
print(causality(var_macro, cause = "inflation")$Granger)

# インパルス応答
irf_macro <- irf(var_macro, n.ahead = 20, boot = TRUE, ci = 0.95)
plot(irf_macro)

cat("\n解釈:\n")
cat("- 金利とインフレの間には双方向の因果関係が存在する可能性\n")
cat("- 金利の上昇はインフレを抑制する効果がある（数四半期のラグを伴う）\n")
cat("- インフレの上昇は金利の引き上げにつながる（中央銀行の反応）\n")

---
## まとめ

この章では以下の内容を学びました：

1. **動的因果効果**: 政策変更の効果が時間を通じて広がる現象
2. **分布ラグモデル（DL）**: 説明変数の過去の値の効果を推定
3. **ADLモデル**: 被説明変数のラグも含めたモデル
4. **乗数の計算**: 衝撃乗数、動的乗数、累積乗数、長期乗数
5. **外生性の仮定**: 厳密外生性とその検定
6. **HAC標準誤差**: 系列相関に頑健な推論
7. **VARモデル**: 複数変数間の動的関係の分析
8. **インパルス応答関数**: ショックの伝播の可視化
9. **予測誤差分散分解**: 変動の源泉の分析